<a href="https://colab.research.google.com/github/m10923039/m10923039_hw01/blob/main/test01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input, Dense, Activation,Dropout

讀取檔案

In [ ]:
df_data=pd.read_csv('/content/ObesityDataSet_raw_and_data_sinthetic.csv')

In [116]:
df_x=df_data.drop(['NObeyesdad'],axis=1)
df_y=df_data.loc[:,'NObeyesdad']
df_x['CH2O'].value_counts()
df_y.value_counts()

Obesity_Type_I         351
Obesity_Type_III       324
Obesity_Type_II        297
Overweight_Level_I     290
Overweight_Level_II    290
Normal_Weight          287
Insufficient_Weight    272
Name: NObeyesdad, dtype: int64

# **資料前處理**

## lable encoding

## family_history_with_overweight 超重的家族病史(是/否) <br>
no  -> 0 <br>
yes  -> 1
## SMOKE 抽煙(是/否) <br>
no  -> 0 <br>
yes  -> 1 <br>
## SCC 卡路里消耗監視(是/否)
no  -> 0 <br>
yes  -> 1 <br>
## CAEC 兩餐之間的食物消耗量
No  ->  1 <br>
no  ->  1 <br>
Sometimes  ->  2 <br>
Frequently  ->  3 <br>
Always  ->  4 <br>
## CALC 酒精消耗量
No  ->  1 <br>
no  ->  1 <br>
Sometimes  ->  2 <br>
Frequently  ->  3 <br>
Always  ->  4 <br>
# one hot 欄位
Gender 性別<br>
MTRANS 交通工具<br>

In [111]:
# df_x
df_x.loc()[df_x['family_history_with_overweight']=='no','family_history_with_overweight']=0
df_x.loc()[df_x['family_history_with_overweight']=='yes','family_history_with_overweight']=1

df_x.loc()[df_x['FAVC']=='no','FAVC']=0
df_x.loc()[df_x['FAVC']=='yes','FAVC']=1

df_x.loc()[df_x['SMOKE']=='no','SMOKE']=0
df_x.loc()[df_x['SMOKE']=='yes','SMOKE']=1

df_x.loc()[df_x['SCC']=='no','SCC']=0
df_x.loc()[df_x['SCC']=='yes','SCC']=1

df_x.loc()[df_x['CAEC']=='No','CAEC']=1
df_x.loc()[df_x['CAEC']=='no','CAEC']=1
df_x.loc()[df_x['CAEC']=='Sometimes','CAEC']=2
df_x.loc()[df_x['CAEC']=='Frequently','CAEC']=3
df_x.loc()[df_x['CAEC']=='Always','CAEC']=4

df_x.loc()[df_x['CALC']=='no','CALC']=1
df_x.loc()[df_x['CALC']=='Sometimes','CALC']=2
df_x.loc()[df_x['CALC']=='Frequently','CALC']=3
df_x.loc()[df_x['CALC']=='Always','CALC']=4

df_x[['family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC','CALC']] =df_x[['family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC','CALC']].astype(int)

x = pd.get_dummies(df_x)
y = pd.get_dummies(df_y)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15)

In [115]:
x_train.dtypes

Age                               float64
Height                            float64
Weight                            float64
family_history_with_overweight      int64
FAVC                                int64
FCVC                              float64
NCP                               float64
CAEC                                int64
SMOKE                               int64
CH2O                              float64
SCC                                 int64
FAF                               float64
TUE                               float64
CALC                                int64
Gender_Female                       uint8
Gender_Male                         uint8
MTRANS_Automobile                   uint8
MTRANS_Bike                         uint8
MTRANS_Motorbike                    uint8
MTRANS_Public_Transportation        uint8
MTRANS_Walking                      uint8
dtype: object

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
input_layer = Input(shape=(x_train.shape[1],))
dense_layer_1 =Dense(32,kernel_initializer=tf.keras.initializers.GlorotNormal() ,activation='relu', name ="nl1" )(input_layer)
dense_layer_2 = Dense(16,kernel_initializer=tf.keras.initializers.GlorotNormal(),activation='relu', name ="nl2" )(dense_layer_1)
dense_layer_2 = Dense(10,activation='relu', name ="nl3" )(dense_layer_2)
dense_layer_2 = Dense(10,activation='relu', name ="nl4" )(dense_layer_2)
dense_layer_2 = Dropout(0.5)(dense_layer_2)
output = Dense(y_train.shape[1],activation='softmax')(dense_layer_2)
model = Model(inputs=input_layer, outputs=output)
print(model.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 24)]              0         
_________________________________________________________________
nl1 (Dense)                  (None, 32)                800       
_________________________________________________________________
nl2 (Dense)                  (None, 16)                528       
_________________________________________________________________
nl3 (Dense)                  (None, 10)                170       
_________________________________________________________________
nl4 (Dense)                  (None, 10)                110       
_________________________________________________________________
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 77  

In [104]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=5)
Adam = tf.optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.997, epsilon=1e-08, decay=0.0)
model.compile(optimizer=Adam, 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics = [tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
              )
history = model.fit(x=x_train, y=y_train,  epochs=50,batch_size =5, callbacks=[callback] )

Epoch 1/50
338/338 [==============================] - 1s 2ms/step - loss: 0.8869 - precision_4: 0.7310 - recall_4: 0.4262
Epoch 2/50
338/338 [==============================] - 1s 2ms/step - loss: 0.9114 - precision_4: 0.7941 - recall_4: 0.4562
Epoch 3/50
338/338 [==============================] - 1s 2ms/step - loss: 1.0702 - precision_4: 0.7316 - recall_4: 0.3736
Epoch 4/50
338/338 [==============================] - 1s 2ms/step - loss: 0.9161 - precision_4: 0.7683 - recall_4: 0.4221
Epoch 5/50
338/338 [==============================] - 1s 2ms/step - loss: 0.8301 - precision_4: 0.7667 - recall_4: 0.4664
Epoch 6/50
338/338 [==============================] - 1s 2ms/step - loss: 0.8398 - precision_4: 0.7859 - recall_4: 0.4663
Epoch 7/50
338/338 [==============================] - 1s 2ms/step - loss: 0.9007 - precision_4: 0.7374 - recall_4: 0.4386
Epoch 8/50
338/338 [==============================] - 1s 2ms/step - loss: 0.8805 - precision_4: 0.7652 - recall_4: 0.4539
Epoch 9/50
338/338 [====

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='acc',mode='max', patience=5)
Adam = tf.optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.997, epsilon=1e-08, decay=0.0)
model.compile(optimizer=Adam, 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics = [tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.Accuracy()]
              )
history = model.fit(x=x_train, y=y_train,  epochs=50,batch_size =5, callbacks=[callback] )

**指標F1**

In [105]:
preds = model.evaluate(x = x_test, y = y_test)

F1 = 2 * (preds[1] * preds[2]) / (preds[1] + preds[2])
print(preds)
print(F1)


14/14 [==============================] - 0s 1ms/step - loss: 0.5269 - precision_4: 0.8619 - recall_4: 0.6785
[0.526938259601593, 0.8618618845939636, 0.6784870028495789]
0.7592592713446303


In [ ]:

pd.DataFrame(history.history).plot(figsize = (8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 2.5)
plt.show()


## 數值

In [ ]:
input_layer = Input(shape=(x_train.shape[1],))
dense_layer_1 =Dense(32,kernel_initializer=tf.keras.initializers.GlorotNormal() ,activation='relu', name ="nl1" )(input_layer)
dense_layer_2 = Dense(16,kernel_initializer=tf.keras.initializers.GlorotNormal(),activation='relu', name ="nl2" )(dense_layer_1)
dense_layer_2 = Dense(10,activation='relu', name ="nl3" )(dense_layer_2)
dense_layer_2 = Dense(10,activation='relu', name ="nl4" )(dense_layer_2)
dense_layer_2 = Dropout(0.5)(dense_layer_2)
output = Dense(y_train.shape[1],activation='softmax')(dense_layer_2)
model = Model(inputs=input_layer, outputs=output)
print(model.summary())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input, Dense, Activation,Dropout

df_white = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',sep=";")
# df_red= pd.read_csv('wine-quality/winequality-red.csv',sep=";")
# df= pd.concat([df_red,df_white],axis=0)
df=df_white



In [ ]:
df

In [ ]:
x=df.drop('quality',axis=1)
y=df['quality']
y=pd.get_dummies(y)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15)


x_train = tf.convert_to_tensor(x_train,dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train,dtype=tf.int32)

x_val = tf.convert_to_tensor(x_val,dtype=tf.float32)
y_val = tf.convert_to_tensor(y_val,dtype=tf.int32)

x_test = tf.convert_to_tensor(x_test,dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test,dtype=tf.int32)


In [ ]:

input_layer = Input(shape=(x_train.shape[1],))
dense_layer_1 =Dense(22,kernel_initializer=tf.keras.initializers.GlorotNormal() ,activation='relu', name ="nl1" )(input_layer)
dense_layer_2 = Dense(11,kernel_initializer=tf.keras.initializers.GlorotNormal(),activation='relu', name ="nl2" )(dense_layer_1)
dense_layer_2 = Dense(10,activation='relu', name ="nl3" )(dense_layer_2)
dense_layer_2 = Dense(10,activation='relu', name ="nl4" )(dense_layer_2)
dense_layer_2 = Dropout(0.5)(dense_layer_2)
dense_layer_2 = Dense(10,activation='relu', name ="nl5" )(dense_layer_2)
output = Dense(y_train.shape[1],activation='softmax')(dense_layer_2)
model = Model(inputs=input_layer, outputs=output)

In [ ]:

model.summary()


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=5)
Adam = tf.optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.997, epsilon=1e-08, decay=0.0)
model.compile(optimizer=Adam, 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics = [tf.keras.metrics.PrecisionAtRecall(),tf.keras.metrics.Recall(),tf.keras.metrics.CategoricalAccuracy()]
              )
history = model.fit(x=x_train, y=y_train,  epochs=30,batch_size =5, callbacks=[callback])


In [ ]:
y_predicted = model.predict(x_test)
history.history


# y_predicted = model.predict(x_test)
preds = model.evaluate(x = x_test, y = y_test)
print()
print(preds)



In [ ]:
pd.DataFrame(history.history).plot(figsize = (8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 2.5)
plt.show()